# pyOPAC Usage Examples: Gas Phase and Solution Phase Energy Prediction

This notebook demonstrates how to use the pyOPAC package to predict **gas phase energy**, **solution phase energy**, **dipole moments**, and **solvation free energy** from molecular structures using SOAP descriptors.

## Overview

This example is based on the `examples/1_gas_phase_solution_phase_energy` dataset, where we predict:
- **Gas_Phase_Energy** (eV): Energy of molecule in gas phase
- **Dipole_X, Dipole_Y, Dipole_Z** (Debye): Dipole moment components (vector property)
- **Solution_Phase_Energy** (eV): Energy of molecule in solution
- **Solvation_Free_Energy** (eV): Free energy of solvation

pyOPAC provides:
- **SOAP Descriptors**: Size-invariant, rotationally equivariant molecular descriptors (perfect for dipole moments!)
- **Property Prediction**: Train neural network models to predict molecular properties
- **Multi-target Regression**: Predict multiple properties simultaneously
- **Easy-to-use API**: Simple Python interface for all operations

Let's get started!

## 1. Installation and Import

First, make sure the package is installed:
```bash
pip install -e .
```

Then import the necessary modules:

In [4]:
%pip install -e .

Obtaining file:///C:/Users/james/OneDrive/Documents/Research%20ND-%20First%20Year/My%20Publications/pyOPAC/pyOPAC/New%20Work
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Using cached sparse-0.17.0-py2.py3-none-any.whl.metadata (5.3 kB)
   ---------------------------------------- 0.0/591.2 kB ? eta -:--:--
   -- ------------------------------------ 41.0/591.2 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------  583.7/591.2 kB 7.4 MB/s eta 0:00:01
   ---------------------------------------- 591.2/591.2 kB 7.4 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Import pyOPAC package
import opac

# Print package information
print(f"pyOPAC version: {opac.__version__}")
print(f"Authors: {opac.__author__}")

# Import main functions
from opac import compute_descriptors, get_all_species, read_xyz_files, MoleculeDataset
from opac.models.trainer import train_model, PropertyPredictor, evaluate_model
from opac.utils.logger import get_logger

# Import other necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path
import json

print("\n✓ Package imported successfully!")

pyOPAC version: 0.1.0
Authors: Etinosa Osaro and Yamil Colon

✓ Package imported successfully!


## 2. Reading Molecular Data

We'll use the example data from `examples/1_gas_phase_solution_phase_energy`:
- **Training data**: `train_example.xyz` - Contains multiple molecules for training
- **Test data**: `test_example.xyz` - Contains molecules for testing
- **Target properties**: Gas phase energy, dipole moments, solution phase energy, solvation free energy

In [7]:
# Read molecules from the example XYZ file
# Note: Adjust the path to match your actual data location
example_data_dir = Path("../examples/1_gas_phase_solution_phase_energy")

# Check if example files exist, otherwise use relative paths
if not example_data_dir.exists():
    # Try alternative path
    example_data_dir = Path("examples/1_gas_phase_solution_phase_energy")

if example_data_dir.exists():
    train_xyz_file = example_data_dir / "train.xyz"
    
    if train_xyz_file.exists():
        # Read molecules from XYZ file
        molecules = read_xyz_files(str(example_data_dir))
        print(f"✓ Read {len(molecules)} molecules from {example_data_dir}")
        print(f"\nFirst molecule:")
        print(f"  Formula: {molecules[0].get_chemical_formula()}")
        print(f"  Number of atoms: {len(molecules[0])}")
        print(f"  Species: {molecules[0].get_chemical_symbols()}")
    else:
        print(f"Note: {train_xyz_file} not found. Creating example molecules for demonstration.")
        from ase import Atoms
        molecules = [
            Atoms('H2O', positions=[[0, 0, 0], [0.96, 0, 0], [0, 0.96, 0]]),
            Atoms('CO2', positions=[[0, 0, 0], [1.16, 0, 0], [-1.16, 0, 0]]),
        ]
        print(f"Created {len(molecules)} example molecules for demonstration")
else:
    print(f"Note: {example_data_dir} not found. Creating example molecules for demonstration.")
    from ase import Atoms
    molecules = [
        Atoms('H2O', positions=[[0, 0, 0], [0.96, 0, 0], [0, 0.96, 0]]),
        Atoms('CO2', positions=[[0, 0, 0], [1.16, 0, 0], [-1.16, 0, 0]]),
    ]
    print(f"Created {len(molecules)} example molecules for demonstration")

print(f"\nTotal molecules: {len(molecules)}")

Note: ..\examples\1_gas_phase_solution_phase_energy\train.xyz not found. Creating example molecules for demonstration.
Created 2 example molecules for demonstration

Total molecules: 2


## 3. Computing SOAP Descriptors

SOAP (Smooth Overlap of Atomic Positions) descriptors are ideal for predicting:
- **Gas/Solution Phase Energies**: Capture 3D molecular structure
- **Dipole Moments**: Rotationally equivariant - perfect for vector properties!
- **Solvation Free Energy**: Size-invariant descriptors work for molecules of any size

### Key Features:
- **Size-invariant**: Fixed-size descriptors regardless of molecule size
- **Rotationally equivariant**: Transform predictably with rotations (critical for dipole moments)
- **Translationally invariant**: Same descriptor regardless of position
- **3D structure-aware**: Based on actual atomic positions

### 3.1 Getting All Species

For size-invariant descriptors, we need to use a fixed species list across all molecules:

In [ ]:
# Get all unique species from your dataset
# This ensures fixed-size descriptors across all molecules
all_species = get_all_species(molecules)

print(f"All species found in dataset: {all_species}")
print(f"Number of unique species: {len(all_species)}")

# Save species list for later use (important for test set!)
species_list_file = "species_list.json"
with open(species_list_file, 'w') as f:
    json.dump(all_species, f)
print(f"\n✓ Species list saved to {species_list_file}")
print("  This list will be used for all molecules (training and test) to ensure size-invariance")

### 3.2 Computing SOAP Descriptors

Now let's compute SOAP descriptors for a molecule:

In [ ]:
# Compute SOAP descriptors for the first molecule as an example
# Use fixed species list for size-invariant descriptors
desc = compute_descriptors(
    molecules[0], 
    species=all_species,  # Fixed species list ensures size-invariance
    rcut=6.0,             # Cutoff radius in Angstrom (larger = more neighbors)
    nmax=6,               # Number of radial basis functions (higher = more radial detail)
    lmax=4,               # Maximum angular momentum (higher = more angular detail, better for dipole moments)
    sigma=0.3             # Width of Gaussian smearing (controls locality)
)

print(f"Example molecule: {molecules[0].get_chemical_formula()}")
print(f"Number of atoms: {len(molecules[0])}")
print(f"Number of SOAP descriptors: {len(desc)}")
print(f"\nDescriptor keys (first 10): {list(desc.keys())[:10]}")
print(f"Descriptor values (first 10): {[f'{v:.6f}' for v in list(desc.values())[:10]]}")

# Convert to list for easier handling
descriptor_values = list(desc.values())
print(f"\nDescriptor shape: {len(descriptor_values)} features")
print("\n✓ SOAP descriptors computed successfully!")
print("\nNote: Higher lmax values are better for vector properties like dipole moments!")

## 4. Computing Descriptors for All Molecules

Now let's compute SOAP descriptors for all training molecules. This ensures all molecules have the same descriptor size (size-invariant):

In [ ]:
# Display information about all molecules
print(f"Processing {len(molecules)} molecules...")
print("\nMolecule information:")
for i, mol in enumerate(molecules[:5]):  # Show first 5
    print(f"  Molecule {i+1}: {mol.get_chemical_formula()} ({len(mol)} atoms)")
if len(molecules) > 5:
    print(f"  ... and {len(molecules) - 5} more molecules")

print(f"\nAll species in dataset: {all_species}")
print(f"Using fixed species list for size-invariant descriptors: {all_species}")
print("\n✓ Molecules prepared for descriptor computation!")

In [ ]:
# Compute SOAP descriptors for all molecules with fixed species list
# This ensures size-invariant descriptors (same size for all molecules)
print("Computing SOAP descriptors for all molecules...")
descriptors_list = []

for idx, atoms in enumerate(molecules):
    try:
        # Use fixed species list to ensure size-invariance
        desc = compute_descriptors(atoms, species=all_species)
        desc['mol_id'] = idx
        descriptors_list.append(desc)
        
        if (idx + 1) % 10 == 0 or idx < 3:  # Show progress every 10 molecules and first 3
            print(f"  Molecule {idx+1}/{len(molecules)} ({atoms.get_chemical_formula()}): "
                  f"{len(desc)-1} descriptors")  # -1 for mol_id
    except Exception as e:
        print(f"  Error processing molecule {idx+1}: {e}")

# Convert to DataFrame
df_descriptors = pd.DataFrame(descriptors_list)

# Reorder columns to put mol_id first
cols = ['mol_id'] + [col for col in df_descriptors.columns if col != 'mol_id']
df_descriptors = df_descriptors[cols]

print(f"\n✓ Descriptors computed successfully!")
print(f"DataFrame shape: {df_descriptors.shape} (molecules × features)")
print(f"All molecules have the same descriptor size: {len(df_descriptors.columns) - 1} SOAP features")

# Save descriptors to CSV
descriptors_file = "descriptors.csv"
df_descriptors.to_csv(descriptors_file, index=False)
print(f"\n✓ Descriptors saved to {descriptors_file}")

## 5. Loading Target Properties from DFT Calculations

The target properties come from DFT calculations (see `examples/1_gas_phase_solution_phase_energy/dft_calculation.py`).

The `results_train.dat` file contains:
- **Gas_Phase_Energy** (eV): Energy in gas phase (column 1)
- **Solution_Phase_Energy** (eV): Energy in solution phase (column 2)
- **Solvation_Free_Energy** (eV): Calculated as Solution_Phase_Energy - Gas_Phase_Energy

Note: Dipole moments (Dipole_X, Dipole_Y, Dipole_Z) are not included in this file format. They can be added separately if available.

Let's load the target properties:

In [ ]:
# Load target properties from results_train.dat
# This file contains DFT calculation results (gas phase and solution phase energies)
# Format: Gas_Phase_Energy(eV),Solution_Phase_Energy(eV)
# (Dipole moments may be in a separate file or computed separately)

results_file = Path("results_train.dat")

if results_file.exists():
    # Read the results file (no header, comma-separated)
    df_results = pd.read_csv(results_file, header=None, names=['Gas_Phase_Energy', 'Solution_Phase_Energy'])
    
    print(f"✓ Loaded target properties from {results_file}")
    print(f"Number of molecules: {len(df_results)}")
    print(f"Properties: {list(df_results.columns)}")
    
    # Calculate solvation free energy (difference between solution and gas phase)
    df_results['Solvation_Free_Energy'] = df_results['Solution_Phase_Energy'] - df_results['Gas_Phase_Energy']
    
    # Add molecule IDs
    df_results['mol_id'] = range(len(df_results))
    
    # Reorder columns
    df_targets = df_results[['mol_id', 'Gas_Phase_Energy', 'Solution_Phase_Energy', 'Solvation_Free_Energy']]
    
    print(f"\nEnergy statistics:")
    print(f"  Gas Phase Energy range: [{df_targets['Gas_Phase_Energy'].min():.2f}, {df_targets['Gas_Phase_Energy'].max():.2f}] eV")
    print(f"  Solution Phase Energy range: [{df_targets['Solution_Phase_Energy'].min():.2f}, {df_targets['Solution_Phase_Energy'].max():.2f}] eV")
    print(f"  Solvation Free Energy range: [{df_targets['Solvation_Free_Energy'].min():.4f}, {df_targets['Solvation_Free_Energy'].max():.4f}] eV")
    
    print(f"\nNote: Dipole moments (Dipole_X, Dipole_Y, Dipole_Z) are not in this file.")
    print(f"      They can be computed separately or added if available in another file.")
    
else:
    print(f"Note: {results_file} not found. Creating example target properties.")
    print("In practice, load from results_train.dat (from DFT calculations)")
    
    targets_list = []
    for idx in range(len(molecules)):
        targets_list.append({
            'mol_id': idx,
            'Gas_Phase_Energy': -400.0 - np.random.rand() * 100,  # Example energy in eV
            'Solution_Phase_Energy': -400.0 - np.random.rand() * 100 - 0.1,  # Example soln energy
            'Solvation_Free_Energy': -0.1 - np.random.rand() * 0.1,          # Example solvation energy
        })
    
    df_targets = pd.DataFrame(targets_list)

print(f"\nTarget properties shape: {df_targets.shape}")
print(f"Target columns: {[col for col in df_targets.columns if col != 'mol_id']}")
print("\nFirst few target properties:")
print(df_targets.head())

# Save targets to CSV
targets_output_file = "targets.csv"
df_targets.to_csv(targets_output_file, index=False)
print(f"\n✓ Target properties saved to {targets_output_file}")

In [ ]:
## 6. Preparing Data for Training

Now let's prepare the data for training a multi-target regression model:

In [ ]:
# Prepare data for training
from sklearn.model_selection import train_test_split

# Merge descriptors and targets on mol_id
df = pd.merge(df_descriptors, df_targets, on='mol_id', how='inner')
print(f"✓ Merged descriptors and targets: {df.shape}")

# Identify descriptor and target columns
descriptor_columns = [col for col in df_descriptors.columns if col != 'mol_id']
target_columns = [col for col in df_targets.columns if col != 'mol_id']

print(f"\nDescriptor features: {len(descriptor_columns)}")
print(f"Target properties: {target_columns}")

# Split into train/test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"\nTraining set size: {len(train_df)} molecules")
print(f"Test set size: {len(test_df)} molecules")

# Prepare data for MoleculeDataset
train_descriptors = train_df[descriptor_columns].to_dict('records')
train_targets = train_df[target_columns].to_dict('records')
test_descriptors = test_df[descriptor_columns].to_dict('records')
test_targets = test_df[target_columns].to_dict('records')

print("\n✓ Data prepared for training!")

## 7. Training the Model

Now let's train a neural network to predict all properties simultaneously (multi-target regression):

In [ ]:
# Create datasets
train_dataset = MoleculeDataset(train_descriptors, train_targets)
test_dataset = MoleculeDataset(test_descriptors, test_targets)

print(f"Training dataset size: {len(train_dataset)} molecules")
print(f"Input dimension (SOAP descriptors): {train_dataset.input_dim}")
print(f"Output dimension (target properties): {train_dataset.output_dim}")
print(f"Target properties: {target_columns}")

# Train model
print("\n" + "="*60)
print("Training multi-target regression model...")
print("="*60)

model = train_model(
    train_dataset,
    input_dim=train_dataset.input_dim,
    output_dim=train_dataset.output_dim,
    epochs=100,          # Number of training epochs
    batch_size=32,       # Batch size
    learning_rate=1e-3,  # Learning rate
    hidden_dim=128,      # Hidden layer dimension
    weight_decay=1e-4     # L2 regularization
)

# Save model
import torch
model_file = "saved_models/trained_model.pth"
Path("saved_models").mkdir(exist_ok=True)
torch.save(model.state_dict(), model_file)
print(f"\n✓ Model trained and saved to {model_file}!")

## 8. Evaluating Model Performance

Let's evaluate the model on the test set:

In [ ]:
# Evaluate model on test set
print("\n" + "="*60)
print("Evaluating model on test set...")
print("="*60)

avg_loss, per_target_metrics = evaluate_model(model, test_dataset, batch_size=32)

print(f"\nOverall Test Loss (MSE): {avg_loss:.6f}")
print("\nPer-target performance metrics:")
print("-" * 60)

for metrics in per_target_metrics:
    target_idx = metrics['target_index']
    target_name = target_columns[target_idx]
    print(f"\n{target_name}:")
    print(f"  MSE: {metrics['MSE']:.6f}")
    print(f"  MAE: {metrics['MAE']:.6f}")
    print(f"  R² Score: {metrics['R2_Score']:.6f}")

print("\n" + "="*60)
print("✓ Model evaluation complete!")

## 9. Making Predictions on New Molecules

Now let's see how to make predictions on new molecules (test set):

In [ ]:
# Make predictions on test molecules
import torch
from torch.utils.data import DataLoader

model.eval()
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

predictions = []
targets = []

with torch.no_grad():
    for batch in test_loader:
        inputs = batch['descriptors']
        outputs = model(inputs)
        predictions.append(outputs.numpy())
        targets.append(batch['targets'].numpy())

predictions = np.concatenate(predictions, axis=0)
targets = np.concatenate(targets, axis=0)

print(f"Predictions shape: {predictions.shape} (molecules × properties)")
print(f"Targets shape: {targets.shape}")

# Create predictions DataFrame
predictions_df = pd.DataFrame(predictions, columns=target_columns)
predictions_df.insert(0, 'mol_id', test_df['mol_id'].values)

# Save predictions
predictions_file = "predictions.csv"
predictions_df.to_csv(predictions_file, index=False)
print(f"\n✓ Predictions saved to {predictions_file}")

# Show sample predictions vs actual
print("\nSample predictions vs actual values:")
comparison_df = pd.DataFrame({
    'mol_id': test_df['mol_id'].values
})
for i, target_name in enumerate(target_columns):
    comparison_df[f'{target_name}_actual'] = targets[:, i]
    comparison_df[f'{target_name}_predicted'] = predictions[:, i]
    comparison_df[f'{target_name}_error'] = np.abs(targets[:, i] - predictions[:, i])

print(comparison_df.head(10))
print("\n✓ Predictions complete!")

## 10. Complete Workflow: Step-by-Step with Python and Bash

This section shows the complete workflow using both Python API and command-line tools.

### 10.1 Step 1: Modify Training XYZ Files

**Python Code:**
```python
import os
import shutil
from creating_the_xyz.modify import modify_xyz_file

# Define file paths
input_xyz = "train_example.xyz"
modified_xyz = "train.xyz"
dest_dir = os.path.join("dataset", "training_xyz_files")
dest_file = os.path.join(dest_dir, "train.xyz")

# Modify the training XYZ file
modify_xyz_file(input_xyz, modified_xyz)
print(f"Modified file created: {modified_xyz}")

# Ensure the destination directory exists and copy the file there
os.makedirs(dest_dir, exist_ok=True)
shutil.copy(modified_xyz, dest_file)
print(f"Modified training file copied to {dest_file}")
```

**Bash Command:**
```bash
# Modify XYZ files to add molecule IDs
pyopac-modify-xyz train_example.xyz train.xyz

# Create directory and copy
mkdir -p dataset/training_xyz_files
cp train.xyz dataset/training_xyz_files/
```

### 10.2 Step 2: Preprocess Training Data and Compute SOAP Descriptors

**Python Code:**
```python
import os
import pandas as pd
import json
from opac.data.loader import read_xyz_files
from opac.data.descriptors import compute_descriptors, get_all_species

# Directory with training XYZ files
input_dir = os.path.join("dataset", "training_xyz_files")

# Read molecules from the XYZ files
molecules = read_xyz_files(input_dir)
print(f"Read {len(molecules)} molecules from {input_dir}")

# Get all unique species for fixed-size descriptors (size-invariant)
all_species = get_all_species(molecules)
print(f"All species found: {all_species}")

# Save species list for later use (important for test set!)
species_list_file = "species_list.json"
with open(species_list_file, 'w') as f:
    json.dump(all_species, f)
print(f"Species list saved to {species_list_file}")

# Compute descriptors for each molecule
descriptors_list = []
for idx, atoms in enumerate(molecules):
    try:
        # Use fixed species list for size-invariant descriptors
        desc = compute_descriptors(atoms, species=all_species)
        desc['mol_id'] = idx  # assign molecule ID
        descriptors_list.append(desc)
        if (idx + 1) % 100 == 0:
            print(f"Processed {idx + 1}/{len(molecules)} molecules")
    except Exception as e:
        print(f"Failed to compute descriptors for molecule {idx}: {e}")

# Create DataFrame and reorder columns so that 'mol_id' comes first
df_descriptors = pd.DataFrame(descriptors_list)
cols = ['mol_id'] + [col for col in df_descriptors.columns if col != 'mol_id']
df_descriptors = df_descriptors[cols]

# Save descriptors to CSV
output_csv = os.path.join("dataset", "descriptors.csv")
df_descriptors.to_csv(output_csv, index=False)
print(f"Descriptors saved to {output_csv}")
print(f"Descriptor shape: {df_descriptors.shape} (molecules × features)")
```

**Bash Command:**
```bash
# Preprocess data and compute descriptors
pyopac-preprocess \
  --input-dir dataset/training_xyz_files/ \
  --targets-file dataset/targets.csv \
  --output-descriptors dataset/descriptors.csv

# Or compute descriptors only (without targets)
pyopac-compute-descriptors \
  --input-dir dataset/training_xyz_files/ \
  --output-descriptors dataset/descriptors.csv
```

### 10.3 Step 3: Train Model (Standard)

**Python Code:**
```python
import os
import json
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from opac.data.dataset import MoleculeDataset
from opac.models.trainer import train_model, evaluate_model

# Load descriptors and targets
df_descriptors = pd.read_csv(os.path.join("dataset", "descriptors.csv"))
df_targets = pd.read_csv(os.path.join("dataset", "targets.csv"))

# Ensure 'mol_id' is numeric and merge on 'mol_id'
df_descriptors['mol_id'] = df_descriptors['mol_id'].astype(int)
df_targets['mol_id'] = df_targets['mol_id'].astype(int)
df = pd.merge(df_descriptors, df_targets, on='mol_id')

# Identify descriptor and target columns
descriptor_cols = [col for col in df_descriptors.columns if col != 'mol_id']
target_cols = [col for col in df_targets.columns if col != 'mol_id']

# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_descriptors = train_df[descriptor_cols].to_dict('records')
train_targets = train_df[target_cols].to_dict('records')
test_descriptors = test_df[descriptor_cols].to_dict('records')
test_targets = test_df[target_cols].to_dict('records')

# Create datasets
train_dataset = MoleculeDataset(train_descriptors, train_targets)
test_dataset = MoleculeDataset(test_descriptors, test_targets)

# Get dimensions for model input/output
input_dim = train_dataset.input_dim
output_dim = train_dataset.output_dim

# Train the model
model = train_model(
    dataset=train_dataset,
    input_dim=input_dim,
    output_dim=output_dim,
    epochs=200,
    batch_size=64,
    learning_rate=0.001,
    hidden_dim=512,
    weight_decay=1e-4
)

# Save the trained model and parameters
model_path = os.path.join("saved_models", "trained_model.pth")
os.makedirs(os.path.dirname(model_path), exist_ok=True)
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

model_params = {"input_dim": input_dim, "hidden_dim": 512, "output_dim": output_dim}
params_path = model_path + ".params.json"
with open(params_path, "w") as f:
    json.dump(model_params, f)
print(f"Model parameters saved to {params_path}")

# Evaluate the model on the test set
test_loss, per_target_metrics = evaluate_model(model, test_dataset, batch_size=64)
print(f"Test Loss: {test_loss:.4f}")
for metric in per_target_metrics:
    print(metric)
```

**Bash Command:**
```bash
# Train model with all options
pyopac-train \
  --descriptors-file dataset/descriptors.csv \
  --targets-file dataset/targets.csv \
  --model-output saved_models/trained_model.pth \
  --epochs 200 \
  --batch-size 64 \
  --learning-rate 0.001 \
  --hidden-dim 512 \
  --weight-decay 1e-4 \
  --test-size 0.2
```

### 10.4 Step 3b: Train Model with Hyperparameter Tuning

**Python Code:**
```python
import os
import json
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import itertools
from opac.data.dataset import MoleculeDataset
from opac.models.trainer import train_model, evaluate_model

# Load descriptors and targets
df_descriptors = pd.read_csv(os.path.join("dataset", "descriptors.csv"))
df_targets = pd.read_csv(os.path.join("dataset", "targets.csv"))

# Ensure 'mol_id' is numeric and merge on 'mol_id'
df_descriptors['mol_id'] = df_descriptors['mol_id'].astype(int)
df_targets['mol_id'] = df_targets['mol_id'].astype(int)
df = pd.merge(df_descriptors, df_targets, on='mol_id')

# Identify descriptor and target columns
descriptor_cols = [col for col in df_descriptors.columns if col != 'mol_id']
target_cols = [col for col in df_targets.columns if col != 'mol_id']

# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_descriptors = train_df[descriptor_cols].to_dict('records')
train_targets = train_df[target_cols].to_dict('records')
test_descriptors = test_df[descriptor_cols].to_dict('records')
test_targets = test_df[target_cols].to_dict('records')

# Create datasets
train_dataset = MoleculeDataset(train_descriptors, train_targets)
test_dataset = MoleculeDataset(test_descriptors, test_targets)

# Get dimensions for model input/output
input_dim = train_dataset.input_dim
output_dim = train_dataset.output_dim

# Example hyperparameter grid
param_grid = {
    'learning_rate': [0.0001, 0.001, 0.01],
    'hidden_dim': [128, 256, 512],
    'weight_decay': [0.0, 1e-4, 1e-3]
}

# Hyperparameter tuning via grid search
best_loss = float('inf')
best_params = None
best_model_state = None

# Create combinations of hyperparameters
for lr, hidden, wd in itertools.product(param_grid['learning_rate'],
                                         param_grid['hidden_dim'],
                                         param_grid['weight_decay']):
    print(f"Training with lr={lr}, hidden_dim={hidden}, weight_decay={wd}")

    # Train the model using the current hyperparameters
    model = train_model(
        dataset=train_dataset,
        input_dim=input_dim,
        output_dim=output_dim,
        epochs=100,  # You can adjust the number of epochs
        batch_size=64,
        learning_rate=lr,
        hidden_dim=hidden,
        weight_decay=wd
    )

    # Evaluate the model on the test set
    test_loss, _ = evaluate_model(model, test_dataset, batch_size=32)
    print(f"Validation loss: {test_loss:.4f}")
    
    # If current configuration is better, store its parameters and model state
    if test_loss < best_loss:
        best_loss = test_loss
        best_params = {
            'input_dim': input_dim, 
            'output_dim': output_dim,
            'learning_rate': lr, 
            'hidden_dim': hidden, 
            'weight_decay': wd
        }
        best_model_state = model.state_dict()  # Save the state_dict

print("Best Hyperparameters:")
print(best_params)
print(f"Best Validation Loss: {best_loss:.4f}")

# Save best model
torch.save(best_model_state, "saved_models/best_trained_model.pth")
with open("saved_models/best_trained_model.pth.params.json", "w") as f:
    json.dump(best_params, f)
```

**Bash Command:**
```bash
# Note: Hyperparameter tuning is typically done via Python scripts
# You can create a script and run it:
python notebooks/03_Train_Model_with_hyperparameters.py
```

### 10.5 Step 4: Modify Test XYZ Files

**Python Code:**
```python
import os
import shutil
from creating_the_xyz.modify import modify_xyz_file

# Define file paths for test XYZ
input_test_xyz = "test_example.xyz"
modified_test_xyz = "test.xyz"
dest_dir = os.path.join("dataset", "testing_xyz_files")
dest_file = os.path.join(dest_dir, "test.xyz")

# Modify the test XYZ file
modify_xyz_file(input_test_xyz, modified_test_xyz)
print(f"Modified test XYZ file created: {modified_test_xyz}")

# Copy the modified file to the testing directory
os.makedirs(dest_dir, exist_ok=True)
shutil.copy(modified_test_xyz, dest_file)
print(f"Modified test file copied to {dest_file}")
```

**Bash Command:**
```bash
# Modify test XYZ files
pyopac-modify-xyz test_example.xyz test.xyz

# Create directory and copy
mkdir -p dataset/testing_xyz_files
cp test.xyz dataset/testing_xyz_files/
```

### 10.6 Step 5: Compute Test Descriptors

**Python Code:**
```python
import os
import pandas as pd
import json
from opac.data.loader import read_xyz_files
from opac.data.descriptors import compute_descriptors

# Load species list from training (CRITICAL for size-invariance!)
species_list_file = "species_list.json"
with open(species_list_file, 'r') as f:
    all_species = json.load(f)
print(f"Using fixed species list: {all_species}")

# Directory containing test XYZ files
input_dir = os.path.join("dataset", "testing_xyz_files")

# Read molecules from the test XYZ files
molecules = read_xyz_files(input_dir)
print(f"Read {len(molecules)} molecules from {input_dir}")

# Compute descriptors for each test molecule (using same species list!)
descriptors_list = []
for idx, atoms in enumerate(molecules):
    try:
        # Use the SAME species list as training for size-invariance
        desc = compute_descriptors(atoms, species=all_species)
        desc['mol_id'] = idx
        descriptors_list.append(desc)
    except Exception as e:
        print(f"Failed to compute descriptors for molecule {idx}: {e}")

# Create a DataFrame and reorder columns so that 'mol_id' comes first
df_new = pd.DataFrame(descriptors_list)
cols = ['mol_id'] + [col for col in df_new.columns if col != 'mol_id']
df_new = df_new[cols]

# Save new descriptors to CSV
output_csv = os.path.join("dataset", "new_descriptors.csv")
df_new.to_csv(output_csv, index=False)
print(f"New descriptors saved to {output_csv}")
```

**Bash Command:**
```bash
# Compute descriptors for test molecules
pyopac-compute-descriptors \
  --input-dir dataset/testing_xyz_files/ \
  --output-descriptors dataset/new_descriptors.csv
```

### 10.7 Step 6: Predict Properties

**Python Code:**
```python
import os
import json
import pandas as pd
import torch
from opac.data.dataset import MoleculeDataset
from opac.models.trainer import PropertyPredictor

# Load new descriptors
df_new = pd.read_csv(os.path.join("dataset", "new_descriptors.csv"))
descriptor_cols = [col for col in df_new.columns if col != 'mol_id']
descriptors = df_new[descriptor_cols].to_dict("records")
dataset = MoleculeDataset(descriptors, targets=None)

# Load model parameters
model_path = os.path.join("saved_models", "trained_model.pth")
params_path = model_path + ".params.json"
with open(params_path, "r") as f:
    model_params = json.load(f)
input_dim = model_params["input_dim"]
hidden_dim = model_params["hidden_dim"]
output_dim = model_params["output_dim"]

# Initialize and load the model
model = PropertyPredictor(input_dim, hidden_dim, output_dim)
model.load_state_dict(torch.load(model_path))
model.eval()

# Make predictions for each molecule
predictions = []
with torch.no_grad():
    for sample in dataset:
        # Add batch dimension to the tensor
        inputs = sample["descriptors"].unsqueeze(0)
        outputs = model(inputs)
        predictions.append(outputs.numpy().flatten())

# Create a DataFrame with predictions
property_names = [f"Predicted_Property_{i+1}" for i in range(output_dim)]
df_preds = pd.DataFrame(predictions, columns=property_names)
df_preds["mol_id"] = df_new["mol_id"]

# Save predictions to CSV
output_csv = os.path.join("dataset", "predictions.csv")
df_preds.to_csv(output_csv, index=False)
print(f"Predictions saved to {output_csv}")
```

**Bash Command:**
```bash
# Make predictions using trained model
pyopac-predict \
  --model-file saved_models/trained_model.pth \
  --descriptors-file dataset/new_descriptors.csv \
  --predictions-output dataset/predictions.csv
```

### 10.8 Step 7: Active Learning

**Python Code:**
```python
import os
import pandas as pd
import torch
from copy import deepcopy
from opac.data.dataset import MoleculeDataset
from opac.models.trainer import train_model, evaluate_model
from opac.active_learning.predict_with_uncertainty import predict_with_uncertainty
from opac.active_learning.uncertainty_sampling import select_most_uncertain_samples
from opac.utils.logger import get_logger

logger = get_logger(__name__)

# Set hyperparameters for active learning
initial_train_size = 1000
query_size = 5
requested_iterations = 2  # maximum number of active learning iterations
hidden_dim = 128
epochs = 50
batch_size = 32
learning_rate = 1e-3
weight_decay = 0.0

# File paths
descriptors_file = os.path.join("dataset", "descriptors.csv")
targets_file = os.path.join("dataset", "targets.csv")
model_output = os.path.join("saved_models", "al_trained_model.pth")
final_al_training_data = os.path.join("dataset", "final_al_training_data.csv")

# Load the descriptors and targets from CSV files
df_descriptors = pd.read_csv(descriptors_file)
df_targets = pd.read_csv(targets_file)

# Merge the data on 'mol_id'
df = pd.merge(df_descriptors, df_targets, on="mol_id")
descriptor_columns = [col for col in df_descriptors.columns if col != "mol_id"]
target_columns = [col for col in df_targets.columns if col != "mol_id"]

# Initialize the labeled (training) and unlabeled datasets
initial_train_df = df.sample(n=initial_train_size, random_state=42)
unlabeled_df = df.drop(initial_train_df.index).reset_index(drop=True)

logger.info(f"Initial training set size: {len(initial_train_df)}")
logger.info(f"Unlabeled set size: {len(unlabeled_df)}")

# Determine the maximum possible iterations based on available unlabeled data
max_possible_iterations = (len(df) - initial_train_size) // query_size
iterations = min(requested_iterations, max_possible_iterations)
if iterations == 0:
    logger.info("Not enough data for the specified iterations and query size.")
else:
    logger.info(f"Active learning will run for {iterations} iterations.")

# Active Learning Loop
model = None  # No pre-trained model to start with

for iteration in range(iterations):
    logger.info(f"--- Active Learning Iteration {iteration + 1}/{iterations} ---")
    
    # Create training dataset from the current labeled data
    train_descriptors = initial_train_df[descriptor_columns].to_dict("records")
    train_targets = initial_train_df[target_columns].to_dict("records")
    train_dataset = MoleculeDataset(train_descriptors, train_targets)
    
    # Create test dataset using the remaining data
    test_df = df.drop(initial_train_df.index).reset_index(drop=True)
    test_descriptors = test_df[descriptor_columns].to_dict("records")
    test_targets = test_df[target_columns].to_dict("records")
    test_dataset = MoleculeDataset(test_descriptors, test_targets)
    
    # Determine model dimensions
    input_dim = len(descriptor_columns)
    output_dim = len(target_columns)
    
    # Train or continue training the model using the current labeled dataset
    model = train_model(
        dataset=train_dataset,
        input_dim=input_dim,
        output_dim=output_dim,
        epochs=epochs,
        batch_size=batch_size,
        learning_rate=learning_rate,
        hidden_dim=hidden_dim,
        weight_decay=weight_decay
    )
    
    # Evaluate the model on the test set
    test_loss, per_target_metrics = evaluate_model(model, test_dataset, batch_size=batch_size)
    logger.info(f"Iteration {iteration + 1} - Test Loss: {test_loss:.4f}")
    for m in per_target_metrics:
        i = m["target_index"]
        logger.info(f"[Target {i}] MSE={m['MSE']:.4f}, MAE={m['MAE']:.4f}, R2={m['R2_Score']:.4f}")
    
    # If there is no more unlabeled data, stop the loop
    if unlabeled_df.empty:
        logger.info("No more unlabeled samples. Stopping active learning.")
        break
    
    # Use Monte Carlo Dropout to predict on unlabeled data and estimate uncertainty
    unlabeled_descriptors = unlabeled_df[descriptor_columns].to_dict("records")
    unlabeled_dataset = MoleculeDataset(unlabeled_descriptors, targets=None)
    predictions, uncertainties = predict_with_uncertainty(model, unlabeled_dataset, batch_size=batch_size)
    
    # Select the samples with the highest uncertainty
    current_query_size = min(query_size, len(unlabeled_df))
    query_indices = select_most_uncertain_samples(uncertainties, current_query_size)
    
    # Add the queried samples to the labeled dataset
    queried_samples = unlabeled_df.iloc[query_indices]
    initial_train_df = pd.concat([initial_train_df, queried_samples], ignore_index=True)
    
    # Remove the queried samples from the unlabeled dataset
    unlabeled_df = unlabeled_df.drop(queried_samples.index).reset_index(drop=True)
    logger.info(f"Iteration {iteration + 1}: Queried {len(queried_samples)} samples.")

# Save the final active learning model
output_dir = os.path.dirname(model_output)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)
    logger.info(f"Created directory {output_dir}.")
torch.save(model, model_output)
logger.info(f"Active Learning completed. Final model saved to {model_output}")

# Save the final active learning training dataset
initial_train_df.to_csv(final_al_training_data, index=False)
logger.info(f"Final AL training data saved to {final_al_training_data}")
```

**Bash Command:**
```bash
# Run active learning workflow
pyopac-active-learning \
  --initial-descriptors-file dataset/descriptors.csv \
  --initial-targets-file dataset/targets.csv \
  --unlabeled-descriptors-file dataset/unlabeled_descriptors.csv \
  --unlabeled-targets-file dataset/unlabeled_targets.csv \
  --budget 10
```

### 10.9 Step 8: Predict with Active Learning Model

**Python Code:**
```python
import os
import pandas as pd
import torch
from opac.data.dataset import MoleculeDataset
from opac.utils.logger import get_logger

logger = get_logger(__name__)

# Set the paths for the model, the input descriptors, and where to save predictions
model_file = os.path.join("saved_models", "al_trained_model.pth")
descriptors_file = os.path.join("dataset", "new_descriptors.csv")
predictions_output = os.path.join("dataset", "new_predictions.csv")

# The CSV should contain a column `mol_id` plus descriptor columns
df_descriptors = pd.read_csv(descriptors_file)
# Assume the descriptor columns are all columns except 'mol_id'
descriptor_columns = [col for col in df_descriptors.columns if col != "mol_id"]
descriptors = df_descriptors[descriptor_columns].to_dict("records")
print(f"Loaded descriptors for {len(descriptors)} molecules.")

# We create a dataset from the descriptors. Since we're only predicting, no targets are needed
dataset = MoleculeDataset(descriptors, targets=None)

# The model was saved using `torch.save(model, model_file)`, so we load the entire model
model = torch.load(model_file)
model.eval()
print(f"Loaded active learning model from {model_file}")

# For each sample in the dataset, we add a batch dimension to the descriptor tensor,
# run the model, and collect the predictions
predictions = []
with torch.no_grad():
    for sample in dataset:
        # Add batch dimension to the descriptor tensor
        inputs = sample["descriptors"].unsqueeze(0)
        outputs = model(inputs)
        predictions.append(outputs.numpy().flatten())

# Create a DataFrame with the predictions and the corresponding `mol_id`, then save to CSV
if predictions:
    output_dim = predictions[0].shape[0]
    property_names = [f"Predicted_Property_{i+1}" for i in range(output_dim)]
else:
    property_names = []

df_preds = pd.DataFrame(predictions, columns=property_names)
df_preds["mol_id"] = df_descriptors["mol_id"]

df_preds.to_csv(predictions_output, index=False)
logger.info(f"Saved predictions to {predictions_output}")
print(f"Predictions saved to {predictions_output}")
```

**Bash Command:**
```bash
# Predict with active learning model (same as regular prediction)
pyopac-predict \
  --model-file saved_models/al_trained_model.pth \
  --descriptors-file dataset/new_descriptors.csv \
  --predictions-output dataset/al_predictions.csv
```

### 10.10 Additional Command-Line Tools

**Generate Molecules (VAE):**
```bash
# Train VAE and generate new molecule descriptors
pyopac-generate \
  --descriptors-file dataset/descriptors.csv \
  --vae-model-output saved_models/vae_model.pth \
  --generated-descriptors-output dataset/generated_descriptors.csv \
  --num-samples 100 \
  --epochs 100
```

**Get Help for Any Command:**
```bash
# Get help for any command-line tool
pyopac-modify-xyz --help
pyopac-preprocess --help
pyopac-compute-descriptors --help
pyopac-train --help
pyopac-predict --help
pyopac-generate --help
pyopac-active-learning --help
```

## 11. Key Features Summary

### SOAP Descriptors for Gas/Solution Phase Energy Prediction

- **Size-Invariant**: All molecules produce descriptors with the same fixed size (works for any molecule size)
- **Rotationally Equivariant**: Descriptors transform predictably with molecular rotations
- **Translationally Invariant**: Same descriptor regardless of molecular position
- **3D Structure-Aware**: Based on actual atomic positions, capturing molecular geometry

### Important Notes for This Example

1. **Fixed Species List**: Always use the same species list (`all_species`) for all molecules (training and test) to ensure consistent descriptor sizes. This is saved in `species_list.json`.

2. **Descriptor Parameters**:
   - `nmax=6`: Number of radial basis functions (controls radial resolution)
   - `lmax=4`: Maximum angular momentum (higher values better for angular details)
   - `rcut=6.0`: Cutoff radius in Angstrom (neighbors within this distance)
   - `sigma=0.3`: Gaussian smearing width (controls locality)

3. **Multi-Target Prediction**: The model predicts multiple properties simultaneously:
   - Gas_Phase_Energy (scalar) - Energy in gas phase
   - Solution_Phase_Energy (scalar) - Energy in solution phase
   - Solvation_Free_Energy (scalar) - Free energy of solvation (calculated as difference)

4. **Data Format**: The `results_train.dat` file contains:
   - Column 1: Gas_Phase_Energy (eV)
   - Column 2: Solution_Phase_Energy (eV)
   - Solvation_Free_Energy is calculated as the difference

5. **Size-Invariance**: By averaging SOAP descriptors over all atoms, we create a global, fixed-size representation regardless of molecule size. This is critical for handling molecules of different sizes in the same dataset.

## Real-World Application

This example demonstrates predicting:
- **Gas phase energies** from DFT calculations (PySCF)
- **Solution phase energies** from DFT calculations with solvent model (ddCOSMO)
- **Solvation free energies** (calculated from the difference)

See `examples/1_gas_phase_solution_phase_energy/dft_calculation.py` for how to compute these properties using PySCF. The `results_train.dat` file contains the actual DFT calculation results.

## Next Steps

- See `QUICK_START.md` for more examples
- Check `README.md` for complete documentation
- Try the `examples/1_gas_phase_solution_phase_energy/` directory for the actual dataset
- Use `examples/TEST_TODAY/` for a complete pipeline example

## Support

For issues or questions:
- Check the documentation files
- See `examples/1_gas_phase_solution_phase_energy/` for the actual data structure
- Contact: eosaro@nd.edu, ycolon@nd.edu